<a href="https://colab.research.google.com/github/Davidxswang/ML/blob/master/Note_1_TensorFlow_Datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow_datasets as tfds
import tensorflow as tf

import matplotlib.pyplot as plt
import numpy as np

In [0]:
! pwd
# mnist dataset is stored in the path below
#! mkdir /content/drive/My\ Drive/colab/mnist

/content


In [0]:
# To see what datasets are available (included in tfds)
if False:
  tfds.list_builders()

In [0]:
# all the configs
as_supervised = False
# if True, elements in ds_train are tuple, if False, elements are dict


batch_size = None  
# -1, load the full batch into a tuple/dict, ds_train will be a tuple if used with as_supervised=True, a dict if as_supervised=False
# None, ds_train will be tf.data.Dataset, no matter what as_supervised is






# load the data
(ds_train, ds_val, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train[:80%]', 'train[80%:]', 'test'],
    shuffle_files=True,
    as_supervised=as_supervised,     
    with_info=True,
    data_dir='/content/drive/My Drive/colab/mnist',
    batch_size=batch_size
)
print(ds_info)
print('ds_train type is (tuple, dict or tf.data.Dataset): ', type(ds_train))





# take a look at the internal structure of the data
if batch_size == -1:
  # ds_train will be dict or tuple
  if as_supervised:
    print('ds_train will be a tuple (images, labels)')
    print('How many images in ds_train: ', len(ds_train[0]))
    print('Type of elements in ds_train\'s images: ', type(ds_train[0][0]))
    print('Shape of the image: ', ds_train[0][0].shape)
    print('Type of elements in ds_train\'s labels: ', type(ds_train[1][0]))
    print('The value of the label: ', ds_train[1][0].numpy())
  else:
    print('ds_train will be a dict {\'image\': image, \'label\': label}')
    print('Type of ds_train[\'image\']: ', type(ds_train['image']))
    print('Shape of ds_train["image"]: ', ds_train['image'].shape)
    print('Type of ds_train[\'label\']: ', type(ds_train['label']))
    print('Shape of ds_train["label"]: ', ds_train['label'].shape)
elif batch_size is None:
  # ds_train will be a tf.data.Dataset
  ds = ds_train.take(1)
  for data in ds:
    print(type(data))
    if as_supervised:
      # data will be tuple if True
      print(type(data[0]))
      print(data[0].shape)
      print(type(data[1]))
      print(data[1].shape)
    else:
      # data will be tuple if False
      print(list(data.keys()))
      print(type(data['image']))
      print(data['image'].shape)
      print(type(data['label']))
      print(data['label'].shape)

tfds.core.DatasetInfo(
    name='mnist',
    version=3.0.0,
    description='The MNIST database of handwritten digits.',
    homepage='http://yann.lecun.com/exdb/mnist/',
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=70000,
    splits={
        'test': 10000,
        'train': 60000,
    },
    supervised_keys=('image', 'label'),
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann. lecun. com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
    redistribution_info=,
)

ds_train type is (tuple, dict or tf.data.Dataset):  <class 'dict'>
ds_train will be a dict {'image': image, 'label': label}
Type of ds_train['image']:  <class 'tensorflow.python.framework.ops.EagerTensor'>
Shape of ds

<class 'tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter'>
<class 'tuple'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
(28, 28, 1)
<class 'tensorflow.python.framework.ops.EagerTensor'>
()


In [0]:
# data is tuple like
if as_supervised:
  # since True, element will be tuple like
  ds = ds_train.take(1)
  for image, label in tfds.as_numpy(ds):
    # after using tfds.as_numpy, the tf.Tensor -> np.ndarray, tf.data.Dataset -> Generator[np.array]
    print(type(image), type(label), label)



<class 'numpy.ndarray'> <class 'numpy.int64'> 8
